# Invalide model - 211passiv i liv-død model
Forklaring: Grundform 211 er en opsat livrente\
Author:          Peter Pommergård Lind\
Email:           pli@pfa.dk\
Encoding:        utf-8\
CITATIONS: Edlund MVS og PFA Tekniske grundlag 2019

In [3]:
###########
# Libraries
###########
import pandas
import numpy as np

In [4]:
#############
## Funktioner
#############
# Integration
def laplace(a,b,f):
  """Simple laplace"""
  integral = 0
  if a==b:
    return 0
  elif b==(a+1):
    integral +=f(a)/2
    integral +=f(b)/2
  else:
    integral +=f(a)/2
    integral +=f(b)/2
    for i in range(a+1,b):
      integral += f(i)
  return integral
# Find dødeligheds intensitet
def muAd(alder,år):
  "Simple interpolation of muAD by rounding age to nearest int"
  if (alder%1==0):
    mu = intensitet[alder] * (1-trend[alder])**(år-2017)
  else:
    alder = round(alder)
    mu = intensitet[alder] * (1-trend[alder])**(år-2017)
  return(mu)


In [5]:
#hent data
muData = pandas.read_excel(r'./data/muAD.xlsx')
intensitet = muData.Intensitet
trend = muData.Forbedring

## Passiv211
Vi antager personen er født 1/1/1981 og har pensionsalder 64. Derudover er beregningstidspunktet den 1/1/2020.\
$$V(t)=\int_t^\infty\exp\bigg(-\int_t^s r+\mu(\tau,x) d\tau \bigg) 1_{\{t+x\geq 64\}} ds $$

In [7]:
def passiv211a(tegning, u,n,år, r):
    """[Find the expected contributions to "opsat livrente"]
    Args:
      tegning ([int]): [alder ved tegningstidspunkt]
      u ([int]): [første udbetalingsdato]
      n ([int]): [sidste udbetalingsdato]
      år ([int]): year of udbetalingsstart
      r ([float]): [interest rate minus "sikkerhedstillæg"]
    """
    t = u - tegning #lower limit for integral (indicator)
    upper = 110-tegning #upper limit for integral
    def f1(tau):
        # Inner integrand
        return (r+muAd(tau+tegning,år+tau))
    def f2(s):
        #second integrand
        return np.exp(-laplace(t,s,f1))
    return laplace(t,upper,f2)


print("Passiv 211: ", passiv211a(tegning=39,n=110, u=64 ,år=2020, r=0))

Passiv 211:  29.853576061774046


## Aktiv uden præmiefritagelse
Vi antager personen er født 1/1/1981 og har pensionsalder 64. Derudover er beregningstidspunktet den 1/1/2020.\
$$ V(t)=\int_t^{64}\exp(-\int_t^s r+\mu d\tau) ds $$

In [8]:
#####################################
# Aktiv uden præmiefritagelse
# ####################################  
def aktivUPræmie(g,n,år, r):
  """[Find the expected contributions to "opsat livrente"]
  Args:
      g ([int]): [alder ved beregningsdato]
      n ([int]): [sidste udbetalingsdato]
      år ([int]): year of udbetalingsstart
      r ([float]): [interest rate minus "sikkerhedstillæg"]
  """
  def f1(alder):
    # Inner integrand
    return (r+muAd(alder,år+(alder-g))) 
  def f2(tau):
    #second integrand
    return np.exp(-laplace(g,tau,f1))
  return laplace(g,n,f2)

print("aktiv uden præmiefritagelse", aktivUPræmie(g=39,n=64,år=2020, r=0))

aktiv uden præmiefritagelse 24.70238518644664


# Simpel liv-død model - 215 passiv med u17 dødeligheder fra PFA tekniske grundlag
$$ V(t)=\int_t^{64}\exp(-\int_t^s r(\tau)+\mu(\tau) \ d\tau) ds $$


In [ ]:
def passiv215(tegning, u,n,år, r):
    """[Find the expected contributions to "opsat livrente"]
    Args:
      tegning ([int]): [alder ved tegningstidspunkt]
      u ([int]): [første udbetalingsdato]
      n ([int]): [sidste udbetalingsdato]
      år ([int]): year of udbetalingsstart
      r ([float]): [interest rate minus "sikkerhedstillæg"]
    """
    t = tegning-u #lower limit for integral (indicator)
    upper = n-tegning #upper limit for integral
    def f1(tau):
        # Inner integrand
        return (r+muAd(tau+tegning,år+tau))
    def f2(s):
        #second integrand
        return np.exp(-laplace(t,s,f1))
    return laplace(t,upper,f2)


print("Passiv 215: ", passiv215(tegning=39,u=39, n=64, år=2020, r=0))